In [1]:
import os
import pickle
import tweepy
import datetime

In [2]:
def load_api(file='credentials.pkl'):
    cred = pickle.load(open(file, 'rb'))
    auth = tweepy.OAuthHandler(cred['consumer_key'], cred['consumer_secret'])
    auth.set_access_token(cred['access_token'], cred['access_token_secret'])
    api = tweepy.API(auth)
    
    return api

api = load_api()

In [23]:
class StreamListener(tweepy.StreamListener):
    def __init__(self, file, lang, query, tweet_limit=100):
        super(StreamListener, self).__init__()
        self.file = file
        self.lang = '|'.join(lang)
        self.query = '|'.join(query)
        self.tweet_limit = tweet_limit
        self.i = 0
        
    def on_connect(self):
        if not os.path.exists(self.file):
            with open(self.file, 'w') as f:
                f.writelines('time,lang,query,user,text\n')
            
        print(f'---Stream opened ({self.file})---')
        
    def on_status(self, status):
        time = status.created_at - datetime.timedelta(hours=5)
        time = time.strftime("%Y-%m-%d %H:%M:%S")
        text = status.text.replace('"', "'")
        text = f'"{text}"'
        
        with open(self.file, 'a') as f:
            f.writelines(time + ',')
            f.writelines(self.lang + ',')
            f.writelines(self.query + ',')
            f.writelines(status.user.name + ',')
            f.writelines(text)
            f.writelines('\n')
        
        self.i += 1
        if self.i < self.tweet_limit:
            print(f'{self.i}/{self.tweet_limit}', end='\r')
        else:
            print(f'---Got {self.tweet_limit} tweets---')
            return False
        
    def on_error(self, status_code):
        print(f'---Got error: {status_code}---')
        return False

    def on_disconnect(self, notice):
        print('---Stream closed---')        

In [24]:
project = 'test'
folder = 'data'
file = datetime.datetime.now().strftime("%Y%m%d")
file = os.path.join(folder, f'{project}_{file}.csv')
query = ['game', 'games', 'space', 'python', 'java']
lang = ['en']

In [26]:
listener = StreamListener(file, lang, query, tweet_limit=5)
myStream = tweepy.Stream(auth=api.auth, listener=listener)

In [27]:
myStream.filter(follow=None,
                track=query,
                locations=None,
                languages=lang,
                async=False)

---Stream opened (data/test_20170804.csv)---
---Got 5 tweets---


In [29]:
myStream.disconnect()
myStream.running

False

In [31]:
import pandas as pd
pd.read_csv(file)

,time,lang,query,user,text
0,2017-08-04 02:27:20,en,game|games|space|python|java,➵KRISTEN🌵,RT @zoexrain: when I need my space but I also ...
1,2017-08-04 02:27:20,en,game|games|space|python|java,Matthew Herrera,RT @HawaiiNewsNow: L.A. Clippers to play Toron...
2,2017-08-04 02:27:20,en,game|games|space|python|java,S I M I,RT @AnselElgort: When you have 3 new episodes ...
3,2017-08-04 02:27:20,en,game|games|space|python|java,Linda,Level 5 - Video Games Live https://t.co/GzvgQn...
4,2017-08-04 02:27:20,en,game|games|space|python|java,LESLIE,RT @JackedYoTweets: Astronauts are not allowed...
